In [1]:
import pandas as pd
data_raw = pd.read_excel("AAPL.xlsx")
data_raw = data_raw.iloc[2:, :] # remove "NOTES" and "Average Check for Errors"
data_raw = data_raw.reset_index(drop = True)

Select columns to use as features

In [2]:
data = data_raw[["Date", "Wiki Traffic- 1 Day Lag", "Wiki 5day disparity", "Wiki MA3 Move", "Wiki MA5 Move", "Wiki EMA5 Move", "Goog ROC", "Goog MA3", "Goog MA5", "Goog EMA5", "Goog 3day Disparity", "Goog RSI (14 days)", "Price RSI (14 days)", "Target"]]

Remove all NaN rows

In [3]:
data = data.dropna(axis = 0)
data = data.reset_index(drop = True)

Split dataset

In [4]:
X = data.iloc[:, 1:-1]
Y = data.iloc[:, -1]

Train/test split

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X.values, Y.values, test_size = 0.2)

Feature scaling

In [6]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

Reshape X data

In [7]:
import numpy as np
X_train_3D = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test_3D = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

Build the LSTM neural network

In [8]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
model = Sequential() # initialize the RNN
model.add(LSTM(units = 20, return_sequences = True, input_shape = (X_train.shape[1], 1))) # input shape: (number of features, 1)
model.add(LSTM(units = 20, return_sequences = True))
model.add(LSTM(units = 20, return_sequences = True))
model.add(LSTM(units = 20))
model.add(Dense(units = 1))
model.compile(optimizer = "rmsprop", loss = "binary_crossentropy", metrics = ["accuracy"])

Train the neural network

In [9]:
model.fit(X_train_3D, Y_train, epochs = 50, batch_size = 5, shuffle = False)

Epoch 1/50
126/126 [==============================] - 14s 22ms/step - loss: 0.7811 - accuracy: 0.4873
Epoch 2/50
126/126 [==============================] - 3s 23ms/step - loss: 0.7125 - accuracy: 0.5064
Epoch 3/50
126/126 [==============================] - 3s 25ms/step - loss: 0.7082 - accuracy: 0.5175
Epoch 4/50
126/126 [==============================] - 3s 24ms/step - loss: 0.7056 - accuracy: 0.5191
Epoch 5/50
126/126 [==============================] - 3s 22ms/step - loss: 0.7038 - accuracy: 0.5191
Epoch 6/50
126/126 [==============================] - 3s 22ms/step - loss: 0.7025 - accuracy: 0.5223
Epoch 7/50
126/126 [==============================] - 3s 22ms/step - loss: 0.7014 - accuracy: 0.5207
Epoch 8/50
126/126 [==============================] - 3s 22ms/step - loss: 0.7005 - accuracy: 0.5143
Epoch 9/50
126/126 [==============================] - 3s 23ms/step - loss: 0.6997 - accuracy: 0.5127
Epoch 10/50
126/126 [==============================] - 3s 22ms/step - loss: 0.6991 - accur

Make predictions

In [10]:
Y_test_pred_prob = model.predict(X_test_3D)
Y_train_pred_prob = model.predict(X_train_3D)

In [11]:
Y_test_pred = (Y_test_pred_prob >= 0.5).astype(int)
Y_train_pred = (Y_train_pred_prob >= 0.5).astype(int)

Performance

In [12]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
metrics = pd.DataFrame()
metrics["Metric"] = ["Accuracy", "Precision", "Recall", "F1"]
metrics["Train"] = [accuracy_score(Y_train, Y_train_pred),
                   precision_score(Y_train, Y_train_pred),
                   recall_score(Y_train, Y_train_pred),
                   f1_score(Y_train, Y_train_pred)]
metrics["Test"] = [accuracy_score(Y_test, Y_test_pred),
                   precision_score(Y_test, Y_test_pred),
                   recall_score(Y_test, Y_test_pred),
                   f1_score(Y_test, Y_test_pred)]
metrics = metrics.set_index("Metric")
print(metrics)
print("")

              Train      Test
Metric                       
Accuracy   0.474522  0.471338
Precision  0.000000  0.000000
Recall     0.000000  0.000000
F1         0.000000  0.000000



/Users/fabrizio/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/fabrizio/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [13]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(Y_test, Y_test_pred))

[[74  0]
 [83  0]]
